### Training

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_metric
import torch.nn as nn
import torch
from tqdm.auto import tqdm
import random
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from commom import load_jsonl, save_jsonl

from transformers import (
    AdamW,
    get_scheduler,
    BertTokenizer,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorForTokenClassification
)

In [3]:
class NLIDataset(Dataset):
    def __init__(self, data_list, max_length=512, model_name="bert-base-multilingual-cased"):
        self.d_list = data_list
        self.len = len(self.d_list)
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.label2index = {
            'SPAM': 0,
            'EDM': 1,
            'HAM': 2
        }

    def __getitem__(self, index):
        data = self.d_list[index]
        context = data['context']
        label = data['label']
        
        processed_sample = dict()
        processed_sample['labels'] = torch.tensor(self.label2index[label])
        tokenized_input = self.tokenizer(context,
                                         max_length=self.max_length,
                                         padding='max_length', 
                                         truncation=True,
                                         return_tensors="pt")
        
        input_items = {key: val.squeeze() for key, val in tokenized_input.items()}
        processed_sample.update(input_items)
        return processed_sample

    def __len__(self):
        return self.len

In [8]:
train_list = load_jsonl('datasets/train.jsonl')
dev_list = load_jsonl('datasets/dev.jsonl')

Load Jsonl: datasets/train.jsonl



0it [00:00, ?it/s]
1322it [00:00, 13219.41it/s]
3001it [00:00, 15312.23it/s]
4718it [00:00, 16159.26it/s]
6408it [00:00, 16448.60it/s]
8053it [00:00, 16057.64it/s]
9661it [00:00, 16043.60it/s]
11320it [00:00, 16214.45it/s]
12943it [00:00, 16151.01it/s]
14560it [00:00, 16156.59it/s]
16186it [00:01, 16186.86it/s]
17860it [00:01, 16355.03it/s]
19497it [00:01, 16356.94it/s]
21133it [00:01, 16084.23it/s]
22798it [00:01, 16252.15it/s]
24447it [00:01, 16322.04it/s]
26080it [00:01, 16203.01it/s]
27701it [00:01, 15938.71it/s]
29297it [00:01, 15925.30it/s]
30891it [00:01, 15924.30it/s]
32487it [00:02, 15933.94it/s]
34081it [00:02, 15588.84it/s]
35765it [00:02, 15946.72it/s]
37362it [00:02, 15687.84it/s]
38997it [00:02, 15880.35it/s]
40673it [00:02, 16135.34it/s]
42289it [00:02, 16120.65it/s]
43922it [00:02, 16149.04it/s]
45538it [00:02, 15764.59it/s]
47117it [00:02, 15770.31it/s]
48696it [00:03, 15732.62it/s]
50271it [00:03, 15319.11it/s]
51806it [00:03, 15270.26it/s]
53339it [00:03, 15285.73it

Load Jsonl: datasets/dev.jsonl



0it [00:00, ?it/s]
1699it [00:00, 16984.99it/s]
3398it [00:00, 16881.82it/s]
5087it [00:00, 16731.73it/s]
6761it [00:00, 16473.59it/s]
8461it [00:00, 16659.93it/s]
10128it [00:00, 16467.00it/s]
11776it [00:00, 16326.50it/s]
13410it [00:00, 16011.98it/s]
15021it [00:00, 16036.98it/s]
16710it [00:01, 16290.97it/s]
18396it [00:01, 16458.23it/s]
20043it [00:01, 16451.90it/s]
21816it [00:01, 16833.60it/s]
23503it [00:01, 16843.57it/s]
25188it [00:01, 16803.35it/s]
26885it [00:01, 16851.07it/s]
28571it [00:01, 16809.61it/s]
30253it [00:01, 16797.86it/s]
32440it [00:01, 16554.35it/s]


In [9]:
train_dataset = NLIDataset(train_list)
dev_dataset = NLIDataset(dev_list)

In [10]:
train_list[0]

{'index': '237242',
 'md5sum': 'b3155772342dd7122bec1df6c9bd33e1',
 'label': 'EDM',
 'context': 'Read email browser 訂閱 取消 訂閱 Subscribe Unsubscribe'}

In [11]:
train_dataset[0]

{'labels': tensor(1),
 'input_ids': tensor([  101, 37182, 79515, 91597,  7169,  8148,  2737,  5010,  7169,  8148,
         24358, 31505, 47116, 10112, 11038, 12892, 19528, 99590, 11044,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,  

In [12]:
# note: 3 classes
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
if torch.cuda.device_count() >1:
    model = nn.DataParallel(model,device_ids=[0])
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [13]:
train_batch_size=40
learning_rate=2e-5 
train_epochs=5

optimizer = AdamW(model.parameters(), lr=learning_rate)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
dev_dataloader = DataLoader(dev_dataset, shuffle=True, batch_size=train_batch_size)
print(len(train_dataloader))
print(len(dev_dataloader))

6488
811


In [14]:
for batch_index, batch_dict in enumerate(train_dataloader):
    print(batch_dict)
    break

{'labels': tensor([1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 2, 1, 0, 1, 1,
        1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]), 'input_ids': tensor([[ 101, 8809, 8809,  ..., 5949, 4769,  102],
        [ 101, 7860, 3457,  ..., 8127, 2358,  102],
        [ 101, 4459, 2864,  ...,    0,    0,    0],
        ...,
        [ 101, 8148, 7154,  ...,    0,    0,    0],
        [ 101, 6252, 2078,  ...,    0,    0,    0],
        [ 101, 4458, 4333,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [16]:
## 進度條
num_training_steps = train_epochs * len(train_dataloader)
progress_bar = tqdm(range(num_training_steps))

## 設定warmup
lr_scheduler = get_scheduler(
  "linear",
  optimizer=optimizer,
  num_warmup_steps=10,
  num_training_steps=num_training_steps
)

## start training
for epoch in range(train_epochs):
    model.train()
    for batch_index, batch_dict in enumerate(train_dataloader):
        
        input_items = {key: val.to(device) for key, val in batch_dict.items()}
#         del input_items['token_type_ids'] ## bart不需要這個
        
        optimizer.zero_grad()
        outputs = model(**input_items)
        
        loss = outputs.loss
        if torch.cuda.device_count() >1: ##多GPU的情況要對loss求平均
            loss = loss.mean()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        progress_bar.update(1)
        
        if batch_index % 500 ==0:
            print('epoch: ', epoch, '  loss: ', loss)
            
    model.eval()
    predictions = []
    references = []
    with torch.no_grad():
        for batch_index, batch_dict in enumerate(dev_dataloader):
            input_items = {key: val.to(device) for key, val in batch_dict.items()}
            outputs = model(**input_items)

            predictions += outputs.logits.argmax(dim=-1).tolist()
            references += batch_dict['labels'].tolist()

    accuracy = accuracy_score(references, predictions)
    f1 = f1_score(references, predictions,average='macro')
    print('acc: ', accuracy)
    print('f1: ', f1)
    
    ## save model
    save_path = 'models/Mail_Classifier_11/epoch_' + str(epoch+1)
    if torch.cuda.device_count() >1:
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(save_path)
    else:
        model.save_pretrained(save_path)

  0%|          | 0/32440 [00:00<?, ?it/s]

epoch:  0   loss:  tensor(1.0818, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0279, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0179, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0125, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.1076, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0075, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0426, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.1469, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0182, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0012, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.1206, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0386, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch:  0   loss:  tensor(0.0091, device='cuda:0', g

# Test

In [4]:
test_list = load_jsonl('datasets/test.jsonl')
test_dataset = NLIDataset(test_list)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=512)

Load Jsonl: datasets/test.jsonl


32440it [00:01, 17434.15it/s]


In [5]:
# note: 3 classes
model_path = 'models/Mail_Classifier_11/epoch_5'
# model_path = 'models/Mail_Classifier/epoch_5'
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [6]:
model.eval()
predictions = []
references = []
num_steps = len(test_dataloader)
progress_bar = tqdm(range(num_steps))

with torch.no_grad():
    for batch_index, batch_dict in enumerate(test_dataloader):
        input_items = {key: val.to(device) for key, val in batch_dict.items()}
        outputs = model(**input_items)

        predictions += outputs.logits.argmax(dim=-1).tolist()
        references += batch_dict['labels'].tolist()
        progress_bar.update(1)

accuracy = accuracy_score(references, predictions)
f1 = f1_score(references, predictions,average='macro')
print('acc: ', accuracy)
print('f1: ', f1)

  0%|          | 0/64 [00:00<?, ?it/s]

acc:  0.9931565967940814
f1:  0.9879735482311608


In [7]:
print(model_path)
print(classification_report(references, predictions))

models/Mail_Classifier_11/epoch_5
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      3723
           1       1.00      1.00      1.00     21875
           2       0.99      0.99      0.99      6842

    accuracy                           0.99     32440
   macro avg       0.99      0.99      0.99     32440
weighted avg       0.99      0.99      0.99     32440



In [12]:
import pandas as pd

error_list = []
error_context = []
reference_label_text = []
reference_label_class = []
predict_label = []
output = pd.DataFrame()

for num, i in enumerate(references):
    # if i != predictions[num]:
    if i == 0:
        error_list.append(num)
        error_context.append(test_list[num]['context'])
        reference_label_text.append(test_list[num]['label'])
        reference_label_class.append(i)
        predict_label.append(predictions[num])
                
        # print(test_list[num])
        # print("GT : ", references[num])
        # print(predictions[num])
        # break
        # error_context = []
        # predict_label = []
        # reference_label = []
        
print(len(error_list))
print(error_list)

output['index'] = error_list
output['pred_label'] = predict_label
output['ref_class'] = reference_label_class
output['ref'] = reference_label_text
output['context'] = error_context

# output.to_csv('list_classifier_error.csv', encoding = 'utf-8-sig')
output.to_csv('right_spam.csv', encoding = 'utf-8-sig')

    
        # test_dataloader
        # print(num)

3723
[0, 21, 24, 56, 61, 92, 97, 100, 101, 109, 111, 117, 125, 129, 148, 152, 158, 165, 169, 191, 193, 198, 203, 217, 220, 222, 232, 234, 251, 257, 264, 284, 294, 306, 312, 315, 321, 332, 335, 338, 340, 357, 375, 379, 380, 406, 422, 431, 445, 449, 452, 465, 496, 498, 503, 510, 514, 530, 544, 549, 553, 559, 561, 567, 582, 589, 593, 599, 612, 618, 623, 648, 660, 663, 665, 668, 671, 684, 709, 710, 718, 729, 736, 739, 759, 770, 777, 778, 780, 788, 789, 802, 819, 831, 857, 871, 873, 882, 883, 885, 890, 893, 907, 927, 934, 947, 952, 957, 959, 972, 998, 1010, 1024, 1027, 1035, 1039, 1045, 1054, 1070, 1072, 1073, 1085, 1095, 1099, 1101, 1107, 1110, 1111, 1113, 1121, 1123, 1126, 1133, 1138, 1140, 1146, 1148, 1156, 1157, 1162, 1165, 1170, 1171, 1174, 1188, 1196, 1200, 1209, 1213, 1216, 1239, 1267, 1268, 1285, 1291, 1295, 1305, 1323, 1326, 1329, 1344, 1361, 1375, 1380, 1393, 1395, 1396, 1408, 1414, 1416, 1419, 1421, 1422, 1424, 1431, 1435, 1443, 1451, 1458, 1480, 1497, 1503, 1514, 1518, 1526, 153

## inference

In [1]:
import torch.nn as nn
import torch
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from commom import load_jsonl, save_jsonl

from transformers import (
    BartTokenizer,
    BartForSequenceClassification,
    BertTokenizer,
    AutoTokenizer,
    AutoModelForSequenceClassification
)

In [2]:
test_list = load_jsonl('datasets/test.jsonl')

Load Jsonl: datasets/test.jsonl


32440it [00:01, 17159.63it/s]


In [10]:
model = AutoModelForSequenceClassification.from_pretrained('models/Mail_Classifier_10/epoch_5', num_labels=3) 
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")  

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
if torch.cuda.device_count() >1:
    model = nn.DataParallel(model,device_ids=[0,1])
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [11]:
test_num = 10

# subject = test_list[test_num]['subject']
context = test_list[test_num]['context']
label = test_list[test_num]['label']

In [12]:
tokenized_input = tokenizer(context,
                            max_length=512,
                            truncation=True,
                            return_tensors="pt")
class_number = {'SPAM':0, 'EDM':1, 'HAM':2}
model.eval()
with torch.no_grad():
    input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    del input_items['token_type_ids'] ## bart不需要這個
    
    outputs = model(**input_items)
    prediction = outputs.logits.argmax(dim=-1)
    print(type(int(prediction)))
    
    
    # print('主旨: ', subject)
    print('內文: ', context)
    print('label: ', class_number[label], label)
    print('predict: ', int(prediction), list(class_number.keys())[list(class_number.values()).index(int(prediction))])
    
    # if int(prediction) == 0:
    #     print('predict: ham')
    # else:
    #     print('predict: spam')

<class 'int'>
內文:  圖片 顯 現時 按此 下載 文宣 網頁 不想 收到 本行 信用卡 行銷 訊息 按此 登入 網路 銀行 信用卡 專區 進行 行銷 訊息 取消 訂閱 設定 電子 訊息 內容 包括 附件 合作金庫 銀行 股份 有限公司 傳送 電子 訊息 內容 機密性 經由 公司 授權 方可 利用 電子 訊息 指定 收件 任何人 公司 電子 訊息 內容 審閱 傳送 散佈 揭露 重製 指定 收件 通知 並請 刪除 電子 訊息 內容 謝謝您 合作 電子 訊息 內容 變更 網際網路 保證 電子 訊息 內容 完整性 公司 變更 修改 竄改 偽造 電子 訊息 內容 恕 不負 責任 網路 通訊 含有 電腦病毒 收件 應 自行 確認 郵件 內容 損害 公司 恕 負責
label:  1 EDM
predict:  1 EDM


## eval

In [13]:
from sklearn.metrics import classification_report

class_number = {'SPAM':0, 'EDM':1, 'HAM':2}

predictions = []
references = []

for batch_index, batch_dict in enumerate(test_list[:200]):
    input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    outputs = model(**input_items)

    predictions += outputs.logits.argmax(dim=-1).tolist()

    references += [class_number[batch_dict['label']]]
    # print(reference[:10])
print(len(predictions))

accuracy = accuracy_score(references, predictions)
f1 = f1_score(references, predictions,average='macro')
print('acc: ', accuracy)
print('f1: ', f1)

print("----------")
classification_report(references, predictions, target_names=class_number)

200
acc:  0.725
f1:  0.2801932367149758
----------


/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n        SPAM       0.00      0.00      0.00        22\n         EDM       0.72      1.00      0.84       145\n         HAM       0.00      0.00      0.00        33\n\n    accuracy                           0.73       200\n   macro avg       0.24      0.33      0.28       200\nweighted avg       0.53      0.72      0.61       200\n'

In [ ]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_metric
import torch.nn as nn
import torch
from tqdm.auto import tqdm
import random
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from common import load_jsonl, save_jsonl

from transformers import (
    AdamW,
    get_scheduler,
    BartTokenizer,
    BartForSequenceClassification,
    BertTokenizer,
    AutoTokenizer,
    AutoModelForSequenceClassification
#     AutoModelForMaskedLM,
)

In [ ]:
import torch
from transformers import BertConfig, BertForSequenceClassification, BertTokenizerFast

In [ ]:
eval_list = load_jsonl('test-data.jsonl')
eval_dataset = NLIDataset(eval_list)
eval_dataset[0]
test_dataloader = DataLoader(eval_dataset, shuffle=True, batch_size=train_batch_size)

In [ ]:
predictions = []
references = []

for batch_index, batch_dict in enumerate(test_dataloader):
    input_items = {key: val.to(device) for key, val in batch_dict.items()}
    outputs = model(**input_items)

    predictions += outputs.logits.argmax(dim=-1).tolist()
    references += batch_dict['labels'].tolist()

accuracy = accuracy_score(references, predictions)
f1 = f1_score(references, predictions,average='macro')
print('acc: ', accuracy)
print('f1: ', f1)

In [ ]:
num = 3
print("the sentence is ")
print()
print(input_items[3])
outputs = model(**input_items[3])
predictions = outputs.logits.argmax(dim=-1).tolist()
print("predict : ", predictions)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label_list = ["0", "1"]
tokenizer = BertTokenizerFast.from_pretrained("Transformers Trainer")
config = BertConfig.from_pretrained("Transformers Trainer", finetuning_task="cola")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

In [ ]:
sentence = "The probable hostile German reaction is unfortunate."  # @param {type:"string"}
tokenized_input = tokenizer(sentence, return_tensors="pt").to(device)
outputs = model(**tokenized_input)
print(f"Prediction: {label_list[outputs.logits.argmax(dim=-1).item()]}")